# Scrapeo de books to scrap

In [483]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as req


In [484]:
html = req.get('https://books.toscrape.com').content
sopa = bs(html,'html.parser')

In [485]:
pagina = sopa.find('ol', {'class': 'row'})

In [486]:
libros = pagina.find_all('li', {'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
len(libros)

20

In [487]:
libros[0].find('a').attrs['href']


'catalogue/a-light-in-the-attic_1000/index.html'

In [508]:
completar = 'https://books.toscrape.com/'

In [510]:
libros[0].find_all('li', {'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})

[]

In [479]:
link_bueno = completar + libros[0].find('a').attrs['href']
link_bueno

'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

In [488]:
links_libros = []
for libro in libros:
    libro.find('a').attrs['href']
    links_libros.append(libro.find('a').attrs['href'])

In [481]:
links_libros

['catalogue/a-light-in-the-attic_1000/index.html',
 'catalogue/tipping-the-velvet_999/index.html',
 'catalogue/soumission_998/index.html',
 'catalogue/sharp-objects_997/index.html',
 'catalogue/sapiens-a-brief-history-of-humankind_996/index.html',
 'catalogue/the-requiem-red_995/index.html',
 'catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'catalogue/the-black-maria_991/index.html',
 'catalogue/starving-hearts-triangular-trade-trilogy-1_990/index.html',
 'catalogue/shakespeares-sonnets_989/index.html',
 'catalogue/set-me-free_988/index.html',
 'catalogue/scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html',
 'catalogue/rip-it-up-and-start-again_986/index.html',
 'catalogue/our-band-could-be-your-life-scene

In [489]:
htmls = []
for link in links_libros:
    htmls.append(req.get(completar + link).text)

In [493]:
sopas1 = []
for html in htmls:
    sopas1.append(bs(html,'html.parser'))

In [504]:
lista = []
for e in sopas1:
    dicc = {}    
#Titulo
    titulo = e.find('h1').text
    dicc['Titulo'] = titulo
#Categoria
    categoria = e.find_all('li')[2].text.replace('\n', '')
    dicc['Categoria'] = categoria
#Precio
    precio = e.find('p', {'class': 'price_color'}).text.split('£')[1]
    dicc['Precio/£'] = precio 
#Esta es stock?
    stock = e.find('p', {'class': 'instock availability'}).text.strip().split('(')[0].strip()
    dicc['Stock?'] = stock
#Cuantos quedan?
    cuantos_quedan = e.find('p', {'class': 'instock availability'}).text.strip().split('(')[1].replace(')','')
    dicc['Quedan'] = cuantos_quedan    
#Descripcion
    descripcion = e.find_all('p')[3].text
    dicc['Descripcion'] = descripcion
#UPC
    upc = e.find('td').text 
    dicc['UPC'] = upc
#Precio sin impuestos
    precio_sin = e.find_all('td')[2].text.split('£')[1]
    dicc['Precio_sin_impuestos/£'] = precio_sin
#Impuestos
    impuestos = e.find_all('td')[4].text.replace('Â', '')
    dicc['Impuestos'] = impuestos
#Valoraciones
    valoraciones = e.find_all('td')[6].text.replace('Â', '')
    dicc['Valoraciones'] = valoraciones
#Portadas
    urls = e.find('img')['src'].split('..')[2]
    dicc['Portadas'] = completar + urls
#Apendeo todas las cosas que tengo para crear la lista de diccionarios
    lista.append(dicc)
#Y creo el df
df =  pd.DataFrame(lista)

In [505]:
#Paso el link a formlato html porque me va a venir mejor para que se vea en el df
df = pd.DataFrame(lista)
df['Portadas'] = df['Portadas'].apply(path_to_image_html)

In [506]:
def mostrar_html(df):
    return display(HTML(df.to_html(escape=False)))


In [507]:
df['Descripcion'] = df['Descripcion'].apply(lambda x: x[:100] + '...' if len(x) > 100 else x)
mostrar_html(df)

,Titulo,Categoria,Precio/£,Stock?,Quedan,Descripcion,UPC,Precio_sin_impuestos/£,Impuestos,Valoraciones,Portadas
0,A Light in the Attic,Poetry,51.77,In stock,22 available,It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and...,a897fe39b1053632,51.77,£0.00,0,
1,Tipping the Velvet,Historical Fiction,53.74,In stock,20 available,"""Erotic and absorbing...Written with starling power.""--""The New York Times Book Review "" Nan King, a...",90fa61229261140a,53.74,£0.00,0,
2,Soumission,Fiction,50.10,In stock,20 available,"Dans une France assez proche de la nÃ´tre, un homme sâengage dans la carriÃ¨re universitaire. Peu ...",6957f44c3847a760,50.10,£0.00,0,
3,Sharp Objects,Mystery,47.82,In stock,20 available,"WICKED above her hipbone, GIRL across her heart Words are like a road map to reporter Camille Preake...",e00eb4fd7b871a48,47.82,£0.00,0,
4,Sapiens: A Brief History of Humankind,History,54.23,In stock,20 available,From a renowned historian comes a groundbreaking narrative of humanityâs creation and evolutionâ...,4165285e1663650f,54.23,£0.00,0,
5,The Requiem Red,Young Adult,22.65,In stock,19 available,Patient Twenty-nine.A monster roams the halls of Soothing Hills Asylum. Three girls dead. 29 is endo...,f77dbf2323deb740,22.65,£0.00,0,
6,The Dirty Little Secrets of Getting Your Dream Job,Business,33.34,In stock,19 available,"Drawing on his extensive experience evaluating applicants for his marketing agency, and featuring st...",2597b5a345f45e1b,33.34,£0.00,0,
7,"The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull",Default,17.93,In stock,19 available,"""If you have a heart, if you have a soul, Karen Hicks' The Coming Woman will make you fall in love w...",e72a5dfc7e9267b2,17.93,£0.00,0,
8,The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics,Default,22.60,In stock,19 available,"For readers of Laura Hillenbrand's Seabiscuit and Unbroken, the dramatic story of the American rowin...",e10e1e165dc8be4a,22.60,£0.00,0,
9,The Black Maria,Poetry,52.15,In stock,19 available,"Praise for Aracelis Girmay:""[Girmay's] every lossâshe calls them estrangementsâis a yearning for...",1dfe412b8ac00530,52.15,£0.00,0,
